In [2]:
import cobra
from cobra.io import load_model

### 模型加载

In [3]:
model = load_model('e_coli_core')

In [4]:
# 模拟胞外条件，培养皿环境
model.boundary

[<Reaction EX_ac_e at 0x7fd4e5fee2f0>,
 <Reaction EX_acald_e at 0x7fd4e5fc77c0>,
 <Reaction EX_akg_e at 0x7fd4e5fc7910>,
 <Reaction EX_co2_e at 0x7fd4e5fee8f0>,
 <Reaction EX_etoh_e at 0x7fd4e5fee920>,
 <Reaction EX_for_e at 0x7fd4e5feea10>,
 <Reaction EX_fru_e at 0x7fd4e5feeb30>,
 <Reaction EX_fum_e at 0x7fd4e5feec50>,
 <Reaction EX_glc__D_e at 0x7fd4e5feed70>,
 <Reaction EX_gln__L_e at 0x7fd4e5feee90>,
 <Reaction EX_glu__L_e at 0x7fd4e5feefb0>,
 <Reaction EX_h_e at 0x7fd4e5fef0d0>,
 <Reaction EX_h2o_e at 0x7fd4e5fef1f0>,
 <Reaction EX_lac__D_e at 0x7fd4e5fef310>,
 <Reaction EX_mal__L_e at 0x7fd4e5fef430>,
 <Reaction EX_nh4_e at 0x7fd4e5fef550>,
 <Reaction EX_o2_e at 0x7fd4e5fef670>,
 <Reaction EX_pi_e at 0x7fd4e5fef790>,
 <Reaction EX_pyr_e at 0x7fd4e5fef8b0>,
 <Reaction EX_succ_e at 0x7fd4e5fef9d0>]

In [5]:
# metabolic reactions
set(model.reactions) - set(model.boundary)

{<Reaction ACALD at 0x7fd4e5f6eb00>,
 <Reaction ACALDt at 0x7fd4e5f9c5e0>,
 <Reaction ACKr at 0x7fd4e5f9cdc0>,
 <Reaction ACONTa at 0x7fd4e5f9d510>,
 <Reaction ACONTb at 0x7fd4e5f9d330>,
 <Reaction ACt2r at 0x7fd4e5f6fbb0>,
 <Reaction ADK1 at 0x7fd4e5f6e2c0>,
 <Reaction AKGDH at 0x7fd4e5f9f040>,
 <Reaction AKGt2r at 0x7fd4e5f6f460>,
 <Reaction ALCD2x at 0x7fd4e5f9c3d0>,
 <Reaction ATPM at 0x7fd4e5f9dc00>,
 <Reaction ATPS4r at 0x7fd4e5f9f1f0>,
 <Reaction BIOMASS_Ecoli_core_w_GAM at 0x7fd4e5f9e950>,
 <Reaction CO2t at 0x7fd4e5f9ff40>,
 <Reaction CS at 0x7fd4e5fc62f0>,
 <Reaction CYTBD at 0x7fd4e5fc6da0>,
 <Reaction D_LACt2 at 0x7fd4e5fc6dd0>,
 <Reaction ENO at 0x7fd4e5fc7a60>,
 <Reaction ETOHt2r at 0x7fd4e5f9fc10>,
 <Reaction FBA at 0x7fd4e5fefaf0>,
 <Reaction FBP at 0x7fd4e5fefc10>,
 <Reaction FORt2 at 0x7fd4e6044340>,
 <Reaction FORt at 0x7fd4e6044220>,
 <Reaction FRD7 at 0x7fd4e60449d0>,
 <Reaction FRUpts2 at 0x7fd4e6044df0>,
 <Reaction FUM at 0x7fd4e6045480>,
 <Reaction FUMt2_2 at 0x

In [5]:
print("exchanges", len(model.exchanges))
print("demands", model.demands)
print("sinks", model.sinks)

exchanges 20
demands []
sinks []


In [52]:
model.reactions[1]

Reaction identifier,PFL
Name,Pyruvate formate lyase
Memory address,0x7f89699bc8b0
Stoichiometry,coa_c + pyr_c --> accoa_c + for_c Coenzyme A + Pyruvate --> Acetyl-CoA + Formate
GPR,((b0902 and b0903) and b2579) or (b0902 and b0903) or (b0902 and b3114) or (b3951 and b3952)
Lower bound,0.0
Upper bound,1000.0


### 反应操作

In [53]:
model.reactions.PFL.reversibility

False

In [56]:
model.reactions[0].check_mass_balance()

{}

In [85]:
model.reactions[0].add_metabolites({model.metabolites.get_by_id("h_c"):-1})
print(model.reactions[0].reaction)
model.reactions[0].subtract_metabolites({model.metabolites.get_by_id("h_c"):-1})
print(model.reactions[0].reaction)
print(model.reactions[0].check_mass_balance())

atp_c + f6p_c --> adp_c + fdp_c
atp_c + f6p_c --> adp_c + fdp_c + h_c
{}


In [138]:
print(model.reactions[0].bounds)
model.reactions[0].bounds = (-10.0, 100.0)
print(model.reactions[0].bounds)
model.reactions[0].bounds = (0.0, 1000.0)
print(model.reactions[0].bounds)

(0.0, 1000.0)
(-10.0, 100.0)
(0.0, 1000.0)


### 代谢物操作

In [87]:
atp = model.metabolites.get_by_id("atp_c")
atp

Metabolite identifier,atp_c
Name,ATP C10H12N5O13P3
Memory address,0x7f89699a5a80
Formula,C10H12N5O13P3
Compartment,c
In 13 reaction(s),"GLNS, PPCK, PGK, BIOMASS_Ecoli_core_w_GAM, PYK, SUCOAS, GLNabc, ACKr, ADK1, PPS, ATPS4r, ATPM, PFK"


In [133]:
print(atp.charge)
print(atp.formula)
print(atp.formula_weight)
print(len(atp.reactions))
for reaction in atp.reactions:
    print(str(reaction).split(':')[0])

-4
C10H12N5O13P3
503.149263
13
GLNS
PPCK
PGK
BIOMASS_Ecoli_core_w_GAM
PYK
SUCOAS
GLNabc
ACKr
ADK1
PPS
ATPS4r
ATPM
PFK


### 基因操作

In [140]:
gpr = model.reactions[0].gpr
print(gpr)
gpr_string = model.reactions[0].gene_reaction_rule
print(gpr_string)

b3916 or b1723
b3916 or b1723


In [149]:
idx = str(model.reactions[0].genes).split(' ')[1]
gene = model.genes.get_by_id(idx)
print(gene)
gene.reactions

b1723


frozenset({<Reaction PFK at 0x7f89699a6fe0>})

In [151]:
for reaction in model.reactions[:5]:
    with model as model:
        reaction.knock_out()
        model.optimize()
        print('%s blocked (bounds: %s), new growth rate %f' %
              (reaction.id, str(reaction.bounds), model.objective.value))

[reaction.bounds for reaction in model.reactions[:5]]

PFK blocked (bounds: (0, 0)), new growth rate 0.704037
PFL blocked (bounds: (0, 0)), new growth rate 0.873922
PGI blocked (bounds: (0, 0)), new growth rate 0.863160
PGK blocked (bounds: (0, 0)), new growth rate 0.000000
PGL blocked (bounds: (0, 0)), new growth rate 0.863813


[(0.0, 1000.0),
 (0.0, 1000.0),
 (-1000.0, 1000.0),
 (-1000.0, 1000.0),
 (0.0, 1000.0)]

In [154]:
with model as model:
    model.objective = 'ATPM'
    print('print objective in first context:', model.objective.expression)
    with model:
        model.objective = 'ACALD'
        print('print objective in second context:', model.objective.expression)
    print('objective after exiting second context:',
          model.objective.expression)

model.objective.expression

print objective in first context: 1.0*ATPM - 1.0*ATPM_reverse_5b752
print objective in second context: 1.0*ACALD - 1.0*ACALD_reverse_fda2b
objective after exiting second context: 1.0*ATPM - 1.0*ATPM_reverse_5b752


1.0*BIOMASS_Ecoli_core_w_GAM - 1.0*BIOMASS_Ecoli_core_w_GAM_reverse_712e5

In [171]:
with model as inner:
    cobra.manipulation.knock_out_model_genes(inner, ["b3916"])
    cobra.manipulation.knock_out_model_genes(inner, ["b1723"])
    print(model.reactions[0].bounds)
print(model.reactions[0].bounds)
model.reactions[0]

(0, 0)
(0.0, 1000.0)


Reaction identifier,PFK
Name,Phosphofructokinase
Memory address,0x7f89699a6fe0
Stoichiometry,"atp_c + f6p_c --> adp_c + fdp_c + h_c ATP C10H12N5O13P3 + D-Fructose 6-phosphate --> ADP C10H12N5O10P2 + D-Fructose 1,6-bisphosphate + H+"
GPR,b3916 or b1723
Lower bound,0.0
Upper bound,1000.0


### 全局配置

In [189]:
cobra_config = cobra.Configuration()
cobra_config.bounds = 10.0, 30.0
cobra_config.solver = "glpk_exact"

In [190]:
model1 = load_model('textbook')

In [192]:
model1.solver
cobra.Reaction("R1", lower_bound=None)

Reaction identifier,R1
Name,
Memory address,0x7f8969423370
Stoichiometry,--> -->
GPR,
Lower bound,10.0
Upper bound,30.0
